In [ ]:
from __init__ import *

In [ ]:
signals = []

for i, dict_ in enumerate(SETTING["model_data_dicts"]):

    signal = pd.read_csv("../output/signal/{}.tsv".format(i), sep="\t", index_col=0,)

    signal.columns.name = dict_["axis_1_name"]

    signals.append(signal)

In [ ]:
if SETTING["model_mode"] == "range":

    axis = 0

    mode = "hs"

elif SETTING["model_mode"] == "deep":

    axis = 1

    mode = "ws"

signals = kraft.dataframe.sync_axis(signals, axis)

In [ ]:
for r in SETTING["mf_rs"]:

    ws, hs, errors = kraft.matrix_factorization.factorize_matrix(
        tuple(signal.to_numpy() for signal in signals), mode, r
    )

    template = "Factor {}.{{}}".format(r)

    factors = pd.Index((template.format(i) for i in range(r)), name="Factor")

    ws = tuple(
        pd.DataFrame(data=w, index=signal.index, columns=factors)
        for w, signal in zip(ws, signals)
    )

    hs = tuple(
        pd.DataFrame(data=h, index=factors, columns=signal.columns)
        for h, signal in zip(hs, signals)
    )

    directory_path = "../output/mf_r{}/".format(r)

    kraft.path.path(directory_path)

    template = "{}/{{}}_{{}}.tsv".format(directory_path)

    for i, w in enumerate(ws):

        w.to_csv(template.format("w", i), sep="\t")

    for i, h in enumerate(hs):

        h.to_csv(template.format("h", i), sep="\t")

    kraft.matrix_factorization.plot_matrix_factorization(
        ws, hs, errors, directory_path=directory_path
    )